# Low Pass Filter in Java

Define the file to read, this one is a Comma-Separated-Values (CSV) file.

In [2]:
final String DATA_FILE = "../../Project.Trunk/REST.clients/TCP.Watch.01/max.gust.csv";

Define a list to hold the `gusts` values we will tread in the data file

In [4]:
List<Double> gusts = new ArrayList<>();
BufferedReader br = new BufferedReader(new FileReader(DATA_FILE));

Read loop. We read the data, and turn the 3rd value into a `double`, stored in the list.

In [7]:
String line = "";
long lineNo = 0;
while (line != null) {
  line = br.readLine();
  if (line != null) {
    lineNo += 1;
    if (lineNo > 1) {
      String[] items = line.split(",");
      if (items.length > 2) {
        String gust = items[2];
        gusts.add(Double.parseDouble(gust));
      }
    }
  }
}
br.close();

We have the data, let's count them, and then proceed.

In [8]:
System.out.println(String.format("We read %d line(s), %d entry(ies) in the buffer.", lineNo, gusts.size()));

We read 1005 line(s), 1004 entry(ies) in the buffer.


In [13]:
double minGust = gusts.stream().min(Comparator.comparing(Double::doubleValue)).get().doubleValue();
double maxGust = gusts.stream().max(Comparator.comparing(Double::doubleValue)).get().doubleValue();
System.out.println(String.format("Gusts from %.03f kts to %.03f kts", minGust, maxGust));

Gusts from 0.000 kts to 79.810 kts


Define the filter method:

In [14]:
public final static double ALPHA = 0.015;
public static double lowPass(double alpha, double value, double acc) {
  return (value * alpha) + (acc * (1 - alpha));
}

Compute the filtered list:

In [16]:
List<Double> filteredGusts = new ArrayList<>();
double acc = 0;
gusts.stream().forEach(gust -> {
   acc = lowPass(ALPHA, gust, acc);
   filteredGusts.add(acc);
 });

In [ ]:
double minFGust = filteredGusts.stream().min(Comparator.comparing(Double::doubleValue)).get().doubleValue();
double maxFGust = filteredGusts.stream().max(Comparator.comparing(Double::doubleValue)).get().doubleValue();
System.out.println(String.format("Filtered Gusts from %.03f kts to %.03f kts", minGust, maxGust));

Display both on a graph